In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from torch.utils.data import DataLoader, TensorDataset
"""
Epoch 1/20, Loss: 1.0505
Epoch 2/20, Loss: 0.0362
Epoch 3/20, Loss: 0.0193
Epoch 4/20, Loss: 0.0152
Epoch 5/20, Loss: 0.0137
Epoch 6/20, Loss: 0.0116
Epoch 7/20, Loss: 0.0086
Epoch 8/20, Loss: 0.0107
Epoch 9/20, Loss: 0.0062
Epoch 10/20, Loss: 0.0038
Epoch 11/20, Loss: 0.0088
Epoch 12/20, Loss: 0.0089
Epoch 13/20, Loss: 0.0072
Epoch 14/20, Loss: 0.0006
Epoch 15/20, Loss: 0.0001
Epoch 16/20, Loss: 0.0001
Epoch 17/20, Loss: 0.0113
Epoch 18/20, Loss: 0.0079
Epoch 19/20, Loss: 0.0010
Epoch 20/20, Loss: 0.0005
F1 Score = 0.9839, Accuracy = 0.9844
"""
def load_data_from_file(file='preprocessed_data_cropped.npz'):
    data = np.load(file)
    return data['X'], data['y']

# Load the data
X, y = load_data_from_file()

# Convert y to a 1D array
y = np.argmax(y, axis=1)

# Split the data into training and testing sets
X_train = X[:39209]
y_train = y[:39209]
X_test = X[39209:]
y_test = y[39209:]

# Convert the data to PyTorch tensors
X_train = torch.tensor(np.transpose(X_train, (0, 3, 1, 2)), dtype=torch.float32)  # Transpose the data
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(np.transpose(X_test, (0, 3, 1, 2)), dtype=torch.float32)  # Transpose the data
y_test = torch.tensor(y_test, dtype=torch.long)

# Create a DataLoader for the training data
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Set up the model, loss function, and optimizer
num_classes = len(np.unique(y_train))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the basic residual block
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.downsample:
            residual = self.downsample(residual)
        x += residual
        x = self.relu(x)
        return x

# Define the ResNet architecture
class ModifiedResNet(nn.Module):
    def __init__(self, block, layers, num_classes):
        super(ModifiedResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 64, layers[0])
        self.layer2 = self.make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self.make_layer
        self.layer3 = self.make_layer(block, 256, layers[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        layers = [block(self.in_channels, out_channels, stride, downsample)]
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Create a modified ResNet model for 32x32 images
def modified_resnet18(num_classes):
    return ModifiedResNet(BasicBlock, [2, 2, 2], num_classes)

model = modified_resnet18(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 20
for epoch in range(num_epochs):
    epoch_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}")


# Evaluate the model
with torch.no_grad():
    X_test = X_test.to(device)
    y_test = y_test.to(device)
    outputs = model(X_test)
    _, y_pred = torch.max(outputs, 1)

    f1 = f1_score(y_test.cpu().numpy(), y_pred.cpu().numpy(), average='weighted')
    accuracy = accuracy_score(y_test.cpu().numpy(), y_pred.cpu().numpy())

# Print the F1 score and accuracy
print(f"F1 Score = {f1:.4f}, Accuracy = {accuracy:.4f}")


Epoch 1/20, Loss: 1.0505
Epoch 2/20, Loss: 0.0362
Epoch 3/20, Loss: 0.0193
Epoch 4/20, Loss: 0.0152
Epoch 5/20, Loss: 0.0137
Epoch 6/20, Loss: 0.0116
Epoch 7/20, Loss: 0.0086
Epoch 8/20, Loss: 0.0107
Epoch 9/20, Loss: 0.0062
Epoch 10/20, Loss: 0.0038
Epoch 11/20, Loss: 0.0088
Epoch 12/20, Loss: 0.0089
Epoch 13/20, Loss: 0.0072
Epoch 14/20, Loss: 0.0006
Epoch 15/20, Loss: 0.0001
Epoch 16/20, Loss: 0.0001
Epoch 17/20, Loss: 0.0113
Epoch 18/20, Loss: 0.0079
Epoch 19/20, Loss: 0.0010
Epoch 20/20, Loss: 0.0005
F1 Score = 0.9839, Accuracy = 0.9844


Epoch 1/20, Loss: 1.0505
Epoch 2/20, Loss: 0.0362
Epoch 3/20, Loss: 0.0193
Epoch 4/20, Loss: 0.0152
Epoch 5/20, Loss: 0.0137
Epoch 6/20, Loss: 0.0116
Epoch 7/20, Loss: 0.0086
Epoch 8/20, Loss: 0.0107
Epoch 9/20, Loss: 0.0062
Epoch 10/20, Loss: 0.0038
Epoch 11/20, Loss: 0.0088
Epoch 12/20, Loss: 0.0089
Epoch 13/20, Loss: 0.0072
Epoch 14/20, Loss: 0.0006
Epoch 15/20, Loss: 0.0001
Epoch 16/20, Loss: 0.0001
Epoch 17/20, Loss: 0.0113
Epoch 18/20, Loss: 0.0079
Epoch 19/20, Loss: 0.0010
Epoch 20/20, Loss: 0.0005
F1 Score = 0.9839, Accuracy = 0.9844